In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [22]:
#############################################################
# РО
############################################################
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
import re
import pymysql

def parse_children_page(url):
    response = requests.get(url)
    
    # Перевірка на успішність відповіді
    if response.status_code == 200:
        # Парсимо HTML за допомогою BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Знаходимо елементи, які містять інформацію (з актуальним класом)
        code = soup.find_all(class_="container-fluid passport")  # Актуалізуйте клас, якщо він інший

        # Перебір знайдених елементів у code
        for element in code:
            # Знаходимо ім'я
            nameAge = element.find(class_="passport__title-name")
                       
            if nameAge:
                nameAge = nameAge.get_text(strip=True)
            else:
                nameAge = ""
                
            # passport__title-number
            #id
            id = element.find(class_="passport__title-number").get_text(strip=True)
            number = re.search(r'\d+', id).group()

            if not number:
                number = 0
                
            # Виводимо фото
            photo = element.find(class_="js-lightBoxGallery").find('img').get('src')
            if not photo:
                photo = ""
            #sex
            sex = element.find("ul").find_all("li")[0]
            
            if sex:
                sex = sex.find_all("span")[1].get_text(strip=True)
                if sex=="Жіноча" or sex=="жіноча":
                    sex = "female"
                elif sex=="Чоловіча" or sex=="чоловіча":
                    sex = "male"
                else:
                    sex = "not fill"
            else:
                sex = ""
            #region
            region = element.find("ul").find_all("li")[1]
            if region:
                region = region.find_all("span")[1].get_text(strip=True)
            else:
                region = ""
            
            #siblings
            siblings = element.find("ul").find_all("li")[2]
            
            #отримати кількість
            siblingsAmount = 0
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                siblingsCount = element.find(class_='passport__content-inner--brother').find_all("li")
                siblingsAmount = len(siblingsCount)
            haveSiblings = False
            
            #invalid
            invalid = element.find("ul").find_all("li")[3]
            
            if invalid:
                if invalid.find_all("span")[1].get_text(strip=True) == "є":
                    invalid = True
                else:
                    invalid = False
            
            #form
            form = element.find("ul").find_all("li")[4]
            if form:
                form = form.find_all("span")[1].get_text(strip=True)
            else:
                form = "форма не визначена"
            
            #contacts
            contacts = element.find("ul").find_all("li")[5]
            if contacts:
                contacts = contacts.find_all("span")[1].get_text(strip=True)
            else:
                contacts = "Контакти не визначено"

            print("______________________________________")
            
            name = nameAge.split(", ")[0]
            if len(nameAge.split(" "))>1:
                age = nameAge.split(" ")[1]
            else:
                age = 0
            if not age:
                age = 0
            
            print('name - ', name)
            print('age - ', age)
           # print('sex - ', sex)
           # print('number - ', number)
           # print('photo - ', photo)
           # print('region - ', region)
           # print('siblings - ', haveSiblings)
           # print('howManySiblings - ', siblingsAmount)
           # print('invalid - ', invalid)
           # print('form - ', form)
           # print('contacts - ', contacts)

             #дістати інформацію про братів і сестер
            siblingsInfo = ""
            if (siblings.find_all("span")[1].get_text(strip=True) == "є"):
                haveSiblings = True
                siblingsArr = element.find(class_='passport__content-inner--brother').find_all("li")
                for sibling in siblingsArr:
                    childLink = sibling.find('a').get('href')
                    fullLink = "https://www.msp.gov.ua"+childLink
                    siblingsInfo += fullLink + "\n"
                    #відкрити посилання і дістати інформацію
                    responseSibl = requests.get(fullLink)
                    soupSibl = BeautifulSoup(responseSibl.text, "html.parser")
                    codeSibl = soupSibl.find_all(class_="container-fluid passport")
                    for childSib in codeSibl:
                        nameSibl = childSib.find(class_="passport__title-name")
                        if nameSibl:
                            siblingsInfo += nameSibl.get_text(strip=True)
                                   

            connection = pymysql.connect(
            host="localhost",
            user="root",
            password="root",
            database="bastards"
            )
            
            # Виконуємо запит
            
            try:
                with connection.cursor() as cursor:
                    query = """
                        INSERT INTO adopt_children (id, childName, sex, age, photo_url, region, siblings, siblings_ammount, siblings_info, invalid, form, contacts) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE contacts = VALUES(contacts)
                    """
                    values = (
                        number,
                        name, 
                        sex,
                        age, 
                        photo, 
                        region, 
                        haveSiblings, 
                        siblingsAmount, 
                        siblingsInfo, 
                        invalid, 
                        form, 
                        contacts
                    )
                    cursor.execute(query, values)
                    connection.commit()
            finally:
                connection.close()
            
            
            # Перехід до наступного блоку
            next_step = element.find(class_="btn btn-default passport__footer-next")
            #next_step = 0 #затичка
            if next_step:
                onclick_text = next_step.get("onclick")
                
                # Витягуємо відносний URL із onclick
                start_index = onclick_text.find("'") + 1
                end_index = onclick_text.rfind("'")
                relative_url = onclick_text[start_index:end_index]
                
                # Об'єднуємо з базовим URL
                base_url = "https://www.msp.gov.ua/children/"
                full_url = urljoin(base_url, relative_url)
                
                # Кодуємо для коректного URL
                encoded_url = quote(full_url, safe=":/?=&")
                parse_children_page(encoded_url)
                print("--------------------------------------")
                print("--------------------------------------")
            
    else:
        print("Не вдалося завантажити сторінку")

# Виклик функції з потрібним URL
#parse_children_page("https://www.msp.gov.ua/children/child.php?n=405181&form=*&male=*&age_from=3&age_to=3&region=*&brothers=*&needs=*")
#parse_children_page("https://www.msp.gov.ua/children/child.php?n=429818&form=*&male=*&age_from=9&age_to=9&region=*&brothers=*&needs=*")
#10 yersold women
parse_children_page("https://www.msp.gov.ua/children/child.php?n=373295&form=*&male=%D0%96%D1%96%D0%BD%D0%BE%D1%87%D0%B0&age_from=4&age_to=4&region=*&brothers=*&needs=*")


______________________________________
name -  Богдана
age -  4
______________________________________
name -  Юлія
age -  4
______________________________________
name -  Марія
age -  4
______________________________________
name -  Марія
age -  4
______________________________________
name -  Валерія
age -  4
______________________________________
name -  Ольга
age -  4
______________________________________
name -  Злата
age -  4
______________________________________
name -  Аліна
age -  4
______________________________________
name -  Ульяна
age -  4
______________________________________
name -  Софія
age -  4
______________________________________
name -  Ірина
age -  4
______________________________________
name -  Катерина
age -  4
______________________________________
name -  Аміна
age -  4
______________________________________
name -  Катерина
age -  4
______________________________________
name -  Софія
age -  4
______________________________________
name -  Вероніка
age 

KeyboardInterrupt: 

In [ ]:
######################################
#Creae base if it need
######################################
import pymysql
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bastards"
)

# Виконуємо запит
try:
    with connection.cursor() as cursor:
        query = "CREATE TABLE adopt_children (\
        id INT PRIMARY KEY AUTO_INCREMENT,\
        childName VARCHAR(100) NOT NULL,\
        sex VARCHAR(20),\
        age INT,\
        photo_url VARCHAR(200) NOT NULL,\
        region TEXT,\
        siblings BOOLEAN,\
        siblings_ammount INT(2),\
        siblings_info TEXT,\
        invalid BOOLEAN,\
        form TEXT,\
        contacts TEXT\
        );"
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
finally:
    connection.close()

In [52]:
####################################
#для тестів
####################################
import pymysql

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="bastards"
)

try:
    with connection.cursor() as cursor:
        query = """
            SELECT * FROM children 
            WHERE id = %s
        """
        values = (55559614,)  # Кортеж з одним значенням
        cursor.execute(query, values)
        
        # Отримуємо результати
        ifNumberExist = cursor.fetchall()
        
        if len(ifNumberExist) != 0:
            for row in ifNumberExist:    
                print("test")
        else:
            print("FALSE")

finally:
    connection.close()


FALSE
